In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from PIL import Image
import time
import heapq
import random
import os
from ChessBoard import ChessBoard
import matplotlib.pyplot as plt
%matplotlib inline

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
model_path = "./ass_net_3.0"

##### 加载模型

In [3]:
sess = []
graph = []
for i in range(10):
    graph.append(tf.Graph())
    sess.append(tf.Session(graph = graph[i]))
    with graph[i].as_default():
        saver =tf.train.import_meta_graph(model_path + '/conv_ver_1.ckpt.meta')
        saver.restore(sess[i],tf.train.latest_checkpoint(model_path))

INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt
INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt
INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt
INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt
INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt
INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt
INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt
INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt
INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt
INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt


In [4]:
#提取输入输出接口
xs = []
w1 = []
b1=[]
w2=[]
b2=[]
w3=[]
b3=[]
y = []
for i in range(10):
    xs.append(graph[i].get_tensor_by_name('xs:0'))
    w1.append(graph[i].get_tensor_by_name('w1:0'))
    b1.append(graph[i].get_tensor_by_name('b1:0'))
    w2.append(graph[i].get_tensor_by_name('w2:0'))
    b2.append(graph[i].get_tensor_by_name('b2:0'))
    w3.append(graph[i].get_tensor_by_name('w3:0'))
    b3.append(graph[i].get_tensor_by_name('b3:0'))
    y.append(graph[i].get_tensor_by_name('y:0'))

##### 训练函数

In [5]:
# 网络参数大致
# i 网络序号  sigma 变异程度
def variation(i,sigma=0.002):
    with graph[i].as_default():
        #卷积层1
        sess[i].run(tf.assign(w1[i],sess[i].run(w1[i])+tf.truncated_normal(shape=[3,3,1,8], stddev=sigma)))
        sess[i].run(tf.assign(b1[i],sess[i].run(b1[i])+tf.truncated_normal(shape=[8], stddev=0.1)))
        #卷积层2
        sess[i].run(tf.assign(w2[i],sess[i].run(w2[i])+tf.truncated_normal(shape=[3,3,8,16], stddev=sigma)))
        sess[i].run(tf.assign(b2[i],sess[i].run(b2[i])+tf.truncated_normal(shape=[16], stddev=0.1)))
        #全连接层
        sess[i].run(tf.assign(w3[i],sess[i].run(w3[i])+tf.truncated_normal(shape=[5*5*16, 1],stddev=sigma)))
        sess[i].run(tf.assign(b3[i],sess[i].run(b3[i])+tf.truncated_normal(shape=[1,1], stddev=0.1)))


In [6]:
# AI函数
def solve(board,ai):
    if board.turn == ChessBoard.BLACK:
        max = -9999999
        pair = [-1,-1]
        val = 0
        #遍历每一个格点
        for i in range(0,15):
            for j in range(0,15):
                if board.board[i][j] == ChessBoard.EMPTY:#这个点是空的
                    with graph[ai].as_default():
                        val = sess[ai].run(y[ai],feed_dict={xs[ai]:board.board.reshape(1,15*15)})
                    if val>max:
                        max = val
                        pair = [i,j]
    else:
        min = 9999999
        pair = [-1,-1]
        val = 0
        for i in range(0,15):
            for j in range(0,15):
                if board.board[i][j]== ChessBoard.EMPTY:#这个点是空的
                    with graph[ai].as_default():
                        val = sess[ai].run(y[ai],feed_dict={xs[ai]:board.board.reshape(1,15*15)})
                    if val<min:
                        min = val
                        pair = [i,j]
    return pair,val[0,0]

In [7]:
def printChess(chess):
    print("  ",end="")
    for i in range(15):
        print("%3d" %(i), end="")
    print("")

    for i in range(15):
        print("%2d" %(i), end="")
        for j in range(15):
            if chess[i, j] == 0:
                print("  *", end="")
            elif chess[i, j] == 1:
                print("  o", end="")
            else:
                print("  -", end="")
        print("")

In [8]:
# 环境反馈函数
board = ChessBoard()

def battle(i,j):
    print("battle i,j")
    board.reset()
    
    while True:
        board.judge_Win()
        if board.winner != ChessBoard.EMPTY:
            break
        if board.num >= 225:
            break
        pair = solve(board,i)
        print(pair)
        board.draw_XY(pair[0][0],pair[0][1])
        
        os.system("cls")
        printChess(board.board)
        print(pair[0][0],pair[0][1])
        
    if board.winner ==ChessBoard.BLACK:
        return 1,-1
    elif board.winner == ChessBoard.WHITE:
        return -2,2
    else:
        return 0,0        

##### 正式训练

In [9]:
N = 10 #种群数量

train_path = './temp_net'

score = np.random.randint(1,N,size=N)
print(score)

[1 3 6 6 1 5 3 2 2 9]


In [10]:
# 10代
for epoch in range(10):
    win = heapq.nsmallest(5,range(len(score)),score.take)
    lose = heapq.nlargest(5,range(len(score)),score.take)
    
    print("epoch:%d" %(epoch))
    print("    win:",win)
    print("    lose:",lose)    
    
    # 变异  覆盖掉差的5名
    for i in range(int(N/2)):
        
        #保存胜方
        saver.save(sess[win[i]],train_path+'/conv_ver_1.ckpt')
        #覆盖负方
        graph[lose[i]] = tf.Graph();

        sess[lose[i]] = tf.Session(graph = graph[lose[i]])
        with graph[lose[i]].as_default():
            saver =tf.train.import_meta_graph(train_path + '/conv_ver_1.ckpt.meta')
            saver.restore(sess[lose[i]],tf.train.latest_checkpoint(train_path))
            
            xs[lose[i]] = graph[lose[i]].get_tensor_by_name('xs:0')
            w1[lose[i]] = graph[lose[i]].get_tensor_by_name('w1:0')
            b1[lose[i]] = graph[lose[i]].get_tensor_by_name('b1:0')
            w2[lose[i]] = graph[lose[i]].get_tensor_by_name('w2:0')
            b2[lose[i]] = graph[lose[i]].get_tensor_by_name('b2:0')
            w3[lose[i]] = graph[lose[i]].get_tensor_by_name('w3:0')
            b3[lose[i]] = graph[lose[i]].get_tensor_by_name('b3:0')
            y[lose[i]] = graph[lose[i]].get_tensor_by_name('y:0')
            
        variation(lose[i])
    
    # 对打评分
    score = np.zeros([N])
    for i in range(N):
        print("    judge:",i,end = "")
        # 五局
        fight = [i]
        for j in range(5):
            idx = random.randint(0,N-1)
            while idx in fight:
                idx = random.randint(0,N-1)
               
            fight.append(idx)
            print(idx)
            #对局
            s = battle(i,idx)
            print("    ",s)
            #打分
            score[i] +=s[0]
            score[idx]+=s[1]
            print("(s[0],s[1])",end = "    ")
        print("")            

epoch:0
    win: [0, 4, 7, 8, 1]
    lose: [9, 2, 3, 5, 1]
    judge: 05
battle i,j
([0, 0], 0.83644307)
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
 0  *  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 1  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 2  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 3  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 4  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 5  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 6  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 7  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 8  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 9  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
10  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
11  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
12  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
13  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
14  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
0 0
([0, 1], 0.83644307)
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
 0  *  o  -  -  -  -  -  -  -  -  -  -  -  -  -
 1  - 

([0, 11], 0.88926136)
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
 0  *  o  *  o  *  o  *  o  *  o  *  o  -  -  -
 1  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 2  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 3  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 4  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 5  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 6  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 7  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 8  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 9  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
10  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
11  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
12  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
13  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
14  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
0 11
([0, 12], 0.8936006)
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
 0  *  o  *  o  *  o  *  o  *  o  *  o  *  -  -
 1  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 2  -  -  -  -  -  -  -  -  -  -  -  -  

([1, 7], -3.9845314)
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
 0  *  o  *  o  *  o  *  o  *  o  *  o  *  o  *
 1  o  *  o  *  o  *  o  *  -  -  -  -  -  -  -
 2  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 3  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 4  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 5  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 6  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 7  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 8  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 9  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
10  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
11  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
12  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
13  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
14  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
1 7
([1, 8], -5.6178055)
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
 0  *  o  *  o  *  o  *  o  *  o  *  o  *  o  *
 1  o  *  o  *  o  *  o  *  o  -  -  -  -  -  -
 2  -  -  -  -  -  -  -  -  -  -  -  -  - 

([2, 3], -15.755196)
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
 0  *  o  *  o  *  o  *  o  *  o  *  o  *  o  *
 1  o  *  o  *  o  *  o  *  o  *  o  *  o  *  o
 2  *  o  *  o  -  -  -  -  -  -  -  -  -  -  -
 3  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 4  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 5  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 6  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 7  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 8  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 9  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
10  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
11  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
12  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
13  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
14  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
2 3
([2, 4], -21.408201)
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
 0  *  o  *  o  *  o  *  o  *  o  *  o  *  o  *
 1  o  *  o  *  o  *  o  *  o  *  o  *  o  *  o
 2  *  o  *  o  *  -  -  -  -  -  -  -  - 

([2, 14], -32.50884)
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
 0  *  o  *  o  *  o  *  o  *  o  *  o  *  o  *
 1  o  *  o  *  o  *  o  *  o  *  o  *  o  *  o
 2  *  o  *  o  *  o  *  o  *  o  *  o  *  o  *
 3  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 4  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 5  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 6  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 7  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 8  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 9  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
10  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
11  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
12  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
13  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
14  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
2 14
([3, 0], -39.00955)
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
 0  *  o  *  o  *  o  *  o  *  o  *  o  *  o  *
 1  o  *  o  *  o  *  o  *  o  *  o  *  o  *  o
 2  *  o  *  o  *  o  *  o  *  o  *  o  * 

...

([0, 1], 0.83644307)
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
 0  *  o  -  -  -  -  -  -  -  -  -  -  -  -  -
 1  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 2  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 3  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 4  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 5  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 6  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 7  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 8  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 9  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
10  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
11  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
12  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
13  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
14  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
0 1
([0, 2], 0.8180162)
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
 0  *  o  *  -  -  -  -  -  -  -  -  -  -  -  -
 1  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 2  -  -  -  -  -  -  -  -  -  -  -  -  -  

([0, 12], 0.8936006)
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
 0  *  o  *  o  *  o  *  o  *  o  *  o  *  -  -
 1  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 2  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 3  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 4  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 5  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 6  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 7  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 8  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 9  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
10  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
11  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
12  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
13  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
14  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
0 12
([0, 13], 0.72789395)
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
 0  *  o  *  o  *  o  *  o  *  o  *  o  *  o  -
 1  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 2  -  -  -  -  -  -  -  -  -  -  -  -  

([1, 8], -5.6178055)
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
 0  *  o  *  o  *  o  *  o  *  o  *  o  *  o  *
 1  o  *  o  *  o  *  o  *  o  -  -  -  -  -  -
 2  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 3  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 4  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 5  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 6  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 7  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 8  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 9  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
10  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
11  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
12  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
13  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
14  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
1 8
([1, 9], -5.8168526)
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
 0  *  o  *  o  *  o  *  o  *  o  *  o  *  o  *
 1  o  *  o  *  o  *  o  *  o  *  -  -  -  -  -
 2  -  -  -  -  -  -  -  -  -  -  -  -  - 

([2, 4], -21.408201)
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
 0  *  o  *  o  *  o  *  o  *  o  *  o  *  o  *
 1  o  *  o  *  o  *  o  *  o  *  o  *  o  *  o
 2  *  o  *  o  *  -  -  -  -  -  -  -  -  -  -
 3  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 4  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 5  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 6  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 7  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 8  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
 9  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
10  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
11  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
12  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
13  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
14  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
2 4
([2, 5], -29.361782)
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
 0  *  o  *  o  *  o  *  o  *  o  *  o  *  o  *
 1  o  *  o  *  o  *  o  *  o  *  o  *  o  *  o
 2  *  o  *  o  *  o  -  -  -  -  -  -  - 

KeyboardInterrupt: 

In [ ]:
print(N)
for z in range(100):
    idx = random.randint(0,N-1)
    print(idx)
               